In [3]:
##=================================
##FUNCTION DEFINITION

def log_ret(prices):
    log_ret=np.log(prices/prices.shift(1))
    return log_ret


def cum_ret(prices,strategy=1): #strategy can be the series of predicted signals
    if type(strategy)==int and strategy==1:
        ret=log_ret(prices)
    elif type(strategy)==str and strategy=='sign':
        ret=np.sign(log_ret(prices))*log_ret(prices)
    else:
        ret=strategy.shift(0)*log_ret(prices)
    cum_ret=np.exp(ret.cumsum())
    return cum_ret


def returns_df(prices):
    df=pd.DataFrame(prices)
    df['log_ret']=log_ret(prices)
    df['cum_ret']=cum_ret(prices)
    return df


def ordered_split(array,train_size_pct):
    '''argument=list of arrays that must be split; returns a list of (train, test) tuples for each array'''
    data_len=len(array[0])
    idx=int(train_size_pct*data_len)
    splits=[]
    for i in range(len(array)):
        train=array[i].iloc[:idx]
        test =array[i].iloc[idx:]
        splits.append((train,test))
    return splits

def simple_split(df,train_size_pct):
    idx=int(train_size_pct*len(df))
    train=df.iloc[:idx]
    test =df.iloc[idx:]
    return (train,test)


###still useful?
def strategy(X,Y,train_size):
    full=pd.concat([X,Y])
    idx=int(train_size*len(full))
    
    X_cols=X.shape[1]
    
    X_train=full.iloc[:idx,:X_cols]
    Y_train=full.iloc[:idx, X_cols]
    X_test=full.iloc[idx:, :X_cols]
    Y_test=full.iloc[idx:,X_cols]
    #close_train=full.iloc[:idx, 3]
    #close_test=full.iloc[idx:,3]
    return X_train, Y_train, X_test, Y_test

In [ ]:
#Support and resistance

def sup_res(price, date, window, plot=False):
    from pandas.tseries.offsets import BDay

    subseries=price[date-BDay(window) : date]

    res_idx=scipy.signal.find_peaks(subseries, distance=window)[0][0]
    sup_idx=scipy.signal.find_peaks(-subseries, distance=window)[0][0]

    res=subseries.iloc[res_idx]
    sup=subseries.iloc[sup_idx]
    
    if plot==True:
        plt.figure(figsize=(7,2))
        plt.plot(price)
        plt.axhline(res,color='g')
        plt.axhline(sup, color='r')
        plt.axvline(date,lw=0.5)
        plt.axvline(date-BDay(window),lw=0.3, linestyle='--')
        
    return sup,res



def sup_res_full(price, plot=False):
    from pandas.tseries.offsets import BDay

    #find indices of highest peak and lowest trough:
    res_idx=scipy.signal.find_peaks(price, distance=len(price))[0][0]
    sup_idx=scipy.signal.find_peaks(-price, distance=len(price))[0][0]

    res=price.iloc[res_idx]
    sup=price.iloc[sup_idx]
    
    range=res-sup
    res_tol=res-0.2*range
    sup_tol=sup+0.2*range
    
    if plot==True:
        plt.figure(figsize=(7,2))
        plt.plot(price)
        plt.axhline(res,color='g')
        plt.axhline(sup, color='r')
        plt.fill_between(x=price.index, y1=res_tol,y2=res, color='green',alpha=0.1)
        plt.fill_between(x=price.index, y1=sup_tol,y2=sup, color='red',alpha=0.1)
        plt.axhline(res_tol,color='g',linestyle='--',lw=1)
        plt.axhline(sup_tol, color='r',linestyle='--',lw=1)
        
    return sup,res